- 랭킹 페이지에서 각 뉴스 페이지 하이퍼링크 수집
- 각 페이지에서 기자 이름, 평가 가져오기

! 같은 기사를 방지하기 위해 ID를 부여하거나 필터링하는 기능이 있어야할듯

! 기자가 여러명인 기사 존재

Q. DB 구축해야 하나? 모은 정보를 어떻게 저장할 것인지 의논 필요

In [2]:
# 필요 패키지 설치
!pip install selenium
!pip install webdriver-manager

In [3]:
# 패키지 import
import time
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Chrome option
chrome_options = Options()
chrome_options.add_argument('--headless')  # 브라우저 창을 띄우지 않으려면 주석 해제

# 웹 드라이버 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# 네이버 뉴스 랭킹 페이지로 이동
driver.get('https://news.naver.com/main/ranking/popularDay.naver?mid=etc&sid1=111')

# 페이지가 로드될 때까지 대기
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))


<selenium.webdriver.remote.webelement.WebElement (session="e4c329b7ba4fd595c54f0f5c2a65bcc6", element="f.3B012C4C6FC385B778097750AB97FD37.d.51FFEA342618C90CA7BFA6133DB1A230.e.60")>

In [4]:
# 뉴스 더보기 확장
while True:
    try:
        button = driver.find_element(By.XPATH, '//*[@id="wrap"]/div[4]/button')
        button.click()
    except:
        print('Extend Done')
        break

Extend Done


In [5]:
# 뉴스 랭킹에서 뉴스 링크 및 제목 가져오기
news_link_elements = driver.find_elements(By.XPATH, '//div[@id="wrap"]//ul/li/div/a')

news_link_list = [] # 이후에 큐로 변환하여 기자의 모든 기사들을 큐에 무한으로 넣어서 증식할 수도 있을 듯

for link in news_link_elements:
    href = link.get_attribute('href')
    news_title = link.text.strip()
    
    # 뉴스 기사가 아닌 항목 필터링
    if news_title == '':
        continue
    
    news_link_list.append({
        'title': news_title,
        'url': href
    })


In [6]:
def extract_news_details(driver, url):
    """
    주어진 뉴스 URL에서 신문사, 기자, 평가 등의 정보를 추출하여 반환합니다.
    """
    driver.get(url)
    
    # 페이지가 로드될 때까지 대기
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    time.sleep(0.5)

    newspaper = driver.find_element(By.CSS_SELECTOR, '#ct > div.media_end_head.go_trans > div.media_end_head_top._LAZY_LOADING_WRAP > a > img.media_end_head_top_logo_img.light_type._LAZY_LOADING._LAZY_LOADING_INIT_HIDE').get_attribute('alt')
    
    # SBS 처리 필요 "byline_s" selector 이용하도록 수정, format 변환 필요
    try:
        editor = driver.find_element(By.CSS_SELECTOR, '#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_journalist > a > em').text
    except:
        editor = "N/A"
    try:
        likeit_element = driver.find_element(By.CSS_SELECTOR, '#likeItCountViewDiv > ul').text
        likeit_text = likeit_element.replace(',','')
        likeit_list = likeit_text.split()
        
        emotion = likeit_list[::2] # 평가
        numbers = map(int, likeit_list[1::2]) # 평가

        news_eval = dict(zip(emotion, numbers)) # 뉴스 평가
    except:
        news_eval = "N/A"


    return {
        'newspaper': newspaper,
        'editor': editor,
        'news_url': url,
        'news_eval': news_eval
    }

In [7]:
# 임시 DB 생성 후 DB에 데이터 넣기
db_sample = []
for news in news_link_list:
    news_detail = extract_news_details(driver, news.get('url'))
    db_sample.append(news_detail)

In [8]:
db_sample[:5]

[{'newspaper': '국민일보',
  'editor': '김남중 기자',
  'news_url': 'https://n.news.naver.com/article/005/0001726817?ntype=RANKING',
  'news_eval': {'쏠쏠정보': 89, '흥미진진': 11, '공감백배': 298, '분석탁월': 3, '후속강추': 20}},
 {'newspaper': '국민일보',
  'editor': '권민지 기자',
  'news_url': 'https://n.news.naver.com/article/005/0001726758?ntype=RANKING',
  'news_eval': {'쏠쏠정보': 67, '흥미진진': 11, '공감백배': 6, '분석탁월': 6, '후속강추': 106}},
 {'newspaper': '국민일보',
  'editor': '한웅희 기자',
  'news_url': 'https://n.news.naver.com/article/005/0001726838?ntype=RANKING',
  'news_eval': {'쏠쏠정보': 0, '흥미진진': 0, '공감백배': 1, '분석탁월': 0, '후속강추': 20}},
 {'newspaper': '국민일보',
  'editor': '황인호 기자',
  'news_url': 'https://n.news.naver.com/article/005/0001726835?ntype=RANKING',
  'news_eval': {'쏠쏠정보': 0, '흥미진진': 3, '공감백배': 1, '분석탁월': 2, '후속강추': 0}},
 {'newspaper': '국민일보',
  'editor': '이다연 기자',
  'news_url': 'https://n.news.naver.com/article/005/0001726743?ntype=RANKING',
  'news_eval': {'쏠쏠정보': 5, '흥미진진': 3, '공감백배': 30, '분석탁월': 2, '후속강추': 4}}]

In [10]:
# JSON 파일로 저장
with open('db.json', 'w', encoding='utf-8') as f:
    json.dump(db_sample, f, ensure_ascii=False, indent=4)

print("데이터가 'db.json' 파일로 성공적으로 저장되었습니다.")

데이터가 'db.json' 파일로 성공적으로 저장되었습니다.
